In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q torch sentence_transformers datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator
from sentence_transformers import InputExample
import logging
from datetime import datetime
import os
import json
import math

# train data

In [ ]:
with open('/content/drive/MyDrive/stsb/stsb-fa-train.json', 'r', encoding = 'utf-8-sig') as f:
  train_data = json.load(f)

train_samples = []

for data in train_data:
  train_samples.append(InputExample(texts=[data['sentence1'], data['sentence2']], label=data['score']))

# dev data

In [ ]:
with open('/content/drive/MyDrive/stsb/stsb-fa-dev.json', 'r', encoding = 'utf-8-sig') as f:
  dev_data = json.load(f)

dev_samples = []

for data in dev_data:
  dev_samples.append(InputExample(texts=[data['sentence1'], data['sentence2']], label=data['score']))

# test data

In [ ]:
with open('/content/drive/MyDrive/stsb/stsb-fa-test.json', 'r', encoding = 'utf-8-sig') as f:
  test_data = json.load(f)

test_samples = []

for data in test_data:
  test_samples.append(InputExample(texts=[data['sentence1'], data['sentence2']], label=data['score']))

# Model

In [ ]:
# Define our Cross-Encoder
train_batch_size = 16
num_epochs = 4
model_save_path = "/content/drive/MyDrive/cross_encoder/cross_encoder-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# We use distilroberta-base as base model and set num_labels=1, which predicts a continuous score between 0 and 1
# model = CrossEncoder("xlm-roberta-base", num_labels=1)
model = CrossEncoder("/content/drive/MyDrive/cross_encoder/cross_encoder-2024-02-08_04-07-50", num_labels=1)

In [ ]:
# We wrap train_samples (which is a List[InputExample]) into a pytorch DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

# We add an evaluator, which evaluates the performance during training
evaluator = CECorrelationEvaluator.from_input_examples(dev_samples, name="sts-dev")

In [ ]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up

In [ ]:
# Train the model
model.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/194 [00:00<?, ?it/s]

Iteration:   0%|          | 0/194 [00:00<?, ?it/s]

Iteration:   0%|          | 0/194 [00:00<?, ?it/s]

Iteration:   0%|          | 0/194 [00:00<?, ?it/s]

In [ ]:
##### Load model and eval on test set
model = CrossEncoder(model_save_path)

evaluator = CECorrelationEvaluator.from_input_examples(test_samples, name="sts-test")
evaluator(model)

0.7768303764071328